In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Gothic') # For Windows. PLOT에서 한글 처리 -> 플롯 만드는 창에 넣어야 됨.

plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False # - 기호 안 깨지게
plt.rcParams['font.size'] = 15

import datetime as dt

In [3]:
# ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# from statsmodels.tsa.arima.model import ARIMA  # statsmodel version 1.12
from statsmodels.tsa.arima_model import ARIMA # statsmodel version 1.10.2
# https://www.statsmodels.org/v0.10.2/generated/statsmodels.tsa.arima_model.ARIMAResults.forecast.html?highlight=arima%20forecast#statsmodels.tsa.arima_model.ARIMAResults.forecast

c:\ai\pythonproject\venv\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
!pip install -U pandas-profiling

You should consider upgrading via the 'C:\AI\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
from pandas_profiling import ProfileReport

In [6]:
# 버전 낮추고 한 번 plot 해봐라

In [7]:
# 측정소:중구 데이터:미세먼지 1시간 (하루 치)
# 맨 마지막 거 빼고 학습 -> 맨 마지막 거 예측
# ARIMA로 예측
# stats ver : 낮추기 -> plot

In [8]:
df=pd.read_csv("./lec18_daily_seoul_air_cond_2020.04.csv")

#my_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M") # 없어도됨
df = pd.read_csv('./dataset/daily_seoul_air_cond_2020.04.csv',
                  parse_dates = ['측정일시'],  
#                  date_parser = my_parser
                ) 
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './dataset/daily_seoul_air_cond_2020.04.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df=df[df['측정소명']=='중구'][['측정일시', '측정소명', '미세먼지 1시간(㎍/㎥)']]
# df=df[df['측정소명']=='중구'][['측정일시', '측정소명', '미세먼지1시간(㎍/㎥)']]

In [ ]:
df.head()

In [ ]:
df=df.sort_values(by="측정일시", ascending=True)

In [ ]:
df.index=range(df.shape[0])

In [ ]:
df=df.rename(columns={'미세먼지 1시간(㎍/㎥)':'미세먼지'})

In [ ]:
df.head()

# ARIMA

In [ ]:
plt.figure(figsize = (10,4))
ax = sns.lineplot(x='측정일시',y='미세먼지',data=df)
plt.xticks(rotation = - 45 )
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)
plt.show()

In [ ]:
p, axes = plt.subplots(1,2,figsize=(10,4))
plot_acf(df['미세먼지'], ax=axes[0])
plot_pacf(df['미세먼지'], ax=axes[1])
plt.show()

# AR ,MA?

## diff = 5

In [ ]:
df['미세먼지_diff']=df['미세먼지'].diff(5)

In [ ]:
plt.figure(figsize = (10,4))
ax = sns.lineplot(x=df['측정일시'],y=df['미세먼지_diff'].dropna())
plt.xticks(rotation = - 45 )
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)
plt.show()

In [ ]:
p, axes = plt.subplots(1,2,figsize=(10,4))
plot_acf(df.loc[5:]['미세먼지_diff'], ax=axes[0])
plot_pacf(df.loc[5:]['미세먼지_diff'], ax=axes[1])
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(df['미세먼지'].iloc[:-1], order=(1,1,2))
fit_res = model.fit(trend='nc')
print(fit_res.summary())

In [ ]:
fit_res.plot_predict()

In [ ]:
pred = fit_res.forecast(steps=3)[0]  # [-2.8579221]
print(pred)
미세먼지실제값=df['미세먼지'].iloc[-1]
미세먼지예측=pred
print(f"미세먼지예측: {미세먼지예측} 실제값: {미세먼지실제값}")

In [27]:
# 강사님
# train = pvdf['중구'].iloc[:-5] 
# test  = pvdf['중구'].iloc[-5:]

model = ARIMA(pvdf, order=(1,1,2), freq='H')
fit_res = model.fit(trend='nc')
# pred = fit_res.forecast(steps=3)  # 4,145.1
pred = fit_res.predict(start=1, end=30, typ='levels', dynamic=False)

print(pred[:5])
print(pvdf.dropna()[1:6])

2020-04-01 01:00:00    24.000000
2020-04-01 02:00:00    24.000000
2020-04-01 03:00:00    26.405361
2020-04-01 04:00:00    35.919666
2020-04-01 05:00:00    47.967324
Freq: H, dtype: float64
측정일시
2020-04-01 01:00:00    24
2020-04-01 02:00:00    26
2020-04-01 03:00:00    34
2020-04-01 04:00:00    45
2020-04-01 05:00:00    43
Name: 중구, dtype: int64
